[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/ik-nlp-tutorials/blob/main/notebooks/W6T_Advanced_Prompting_Generation.ipynb)

In [2]:
# Run in Notebook to install local packages
!pip install torch transformers bitsandbytes accelerate rank_bm25 outlines datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

checkpoint = "Qwen/Qwen2.5-1.5B-Instruct"

# Configure 8-bit quantization. We use this to save VRAM, as we don't have a lot available.
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True  # Enables 8-bit quantization
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,  # Apply BitsAndBytesConfig
    device_map="cuda"   # Assign to GPU
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# Load in Sciq dataset
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["validation"])
df_test = pd.read_parquet("hf://datasets/allenai/sciq/" + splits["test"])

# Merge all datasets into one
df_all = pd.concat([df_train, df_val], ignore_index=True)


In [13]:
experiments = ["baseline_answer", "question_context","previous_questions_context", "baseline_answer_A", "question_context_A","previous_questions_context_A"]
prev_row = None
prev2_row = None
original_context = "You are a QA system that only answers with a singular letter as an answer"

for experiment in experiments:
  for idx, row in df_all.iterrows():
    if "_A" in experiment:
      to_ask = (
      row["question"] +
      " A " + row["correct_answer"] +
      ", B " + row["distractor2"] +
      ", C " + row["distractor1"] +
      ", D " + row["distractor3"]
      )
    else:
      to_ask = (
        row["question"] +
        " A " + row["distractor3"] +
        ", B " + row["distractor2"] +
        ", C " + row["distractor1"] +
        ", D " + row["correct_answer"]
      )
    if "n_context" in experiment:
        context = (
            original_context +
            row["support"]
        )
    elif "previous" in experiment:
        context = (
            original_context +
            row["support"] +
            prev_row["support"] +
            prev2_row["support"]
        )

    else:
        context = original_context
    prev2_row = prev_row
    prev_row = row
    messages = [
      {
          "role": "system",
          "content": context,
      },
      {"role": "user", "content": to_ask},
      ]
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    outputs = model.generate(tokenized_chat, max_new_tokens=128)
    answer = tokenizer.decode(outputs[0]).splitlines()[-1][0]
    df_all.loc[idx, experiment] = answer








In [ ]:
import pandas as pd
df = pd.DataFrame(df_all)
df.to_csv("qwen_med.csv")


                                            question        distractor3  \
0  What type of organism is commonly used in prep...            viruses   
1  What phenomenon makes global winds blow northe...    tropical effect   
2  Changes from a less-ordered state to a more-or...        endothermic   
3     What is the least dangerous radioactive decay?         zeta decay   
4  Kilauea in hawaii is the world’s most continuo...              magma   
5  When a meteoroid reaches earth, what is the re...              orbit   
6  What kind of a reaction occurs when a substanc...  nitrogen reaction   
7  Organisms categorized by what species descript...    species complex   
8                  Alpha emission is a type of what?              light   
9          What is the stored food in a seed called?             larval   

          distractor1         distractor2        correct_answer  \
0            protozoa         gymnosperms  mesophilic organisms   
1         muon effect  centrifugal effec